# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/openai/gpt-oss-120b',
 '/openai/gpt-oss-20b',
 '/Qwen/Qwen-Image',
 '/rednote-hilab/dots.ocr',
 '/KittenML/kitten-tts-nano-0.1',
 '/models',
 '/spaces/Qwen/Qwen-Image',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Krea-dev',
 '/spaces/Wan-AI/Wan-2.2-5B',
 '/spaces/alexnasa/OmniAvatar',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/jxm/gpt-oss20b-samples',
 '/datasets/HuggingFaceH4/Multilingual-Thinking',
 '/datasets/openai/BrowseCompLongContext',
 '/datasets/nvidia/Nemotron-Post-Training-Dataset-v1',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/d

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://huggingface.co"))

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [35]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [39]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'company page', 'url': 'https://faruhaikal.wixsite.com/iamfarman'}, {'type': 'resume page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/resume'}, {'type': 'projects page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/s-projects-basic'}, {'type': 'services page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/services-7'}, {'type': 'contact page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/contact'}]}


# Farman Karim - Your Partner in Software Solutions

---

## About Me

Hello! I'm **Farman Karim**, a passionate **Odoo Developer** and **Data Analyst** with over 3 years of experience in delivering efficient software solutions tailored to client needs. I specialize in creating scalable systems and transforming complex datasets into actionable insights that drive business growth and operational efficiency. 

> **My Work, Your Vision** - Let's collaborate and turn your ideas into reality!

---

## Services Offered

- **Custom Odoo Development**: Creating personalized Odoo modules tailored for specific business requirements.
- **Data Analysis & Visualization**: Leveraging tools like Power BI and programming languages such as Python & R to unveil insights that aid decision-making.
- **Project Management**: Handling end-to-end project implementations from requirement gathering to deployment, ensuring the seamless execution of all development stages.

---

## Professional Experience

### **Odoo Developer** - Damotic Technologies (Apr 2025 - Present)
- Developed custom Odoo projects including a comprehensive Parcel Management System and a Luxury Ride Management Platform.

### **Odoo Developer** - SyntecX Limited (Apr 2023 - Mar 2024)
- Led full Odoo implementations, managed development teams, and customized Odoo modules.

### **Odoo Developer** - Galaxy Solutions ITC (May 2021 – Mar 2023)
- Collaborated with cross-functional teams to analyze user needs and enhance the functionality of the Odoo platform.

---

## Education

- **MS in Business Intelligence** (Current) - Université Marie-et-Louis-Pasteur, Besançon, France
- **Bachelor in Computer Science** - University of Management and Technology, Lahore, Pakistan

---

## Skills 

- **Technical**: Odoo Development, Python, JavaScript, PostgreSQL, XML
- **Tools**: Power BI, Docker, GIT, NGINX
- **Languages**: English (Proficient), French (Beginner)

---

## Company Culture

At Farman Karim, I value excellence, collaboration, and continuous learning. Emphasizing a culture that fosters innovation and problem-solving, I am committed to enhancing client satisfaction through customized solutions and skilled project management.

---

## Contact Information

Interested in collaborating or have inquiries? Reach out:

- **Phone**: +33 76 96 214 73
- **Email**: [faruhaikal@gmail.com](mailto:faruhaikal@gmail.com)

---

Join me in unlocking the potential of your visions through technology!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [37]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [40]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'company page', 'url': 'https://faruhaikal.wixsite.com/iamfarman'}, {'type': 'resume page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/resume'}, {'type': 'projects page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/s-projects-basic'}, {'type': 'services page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/services-7'}, {'type': 'contact page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/contact'}]}


# Farman Karim Brochure

---

## About Farman Karim

**Farman Karim** is a dedicated Odoo Developer and Data Analyst with over three years of experience in creating custom Odoo solutions and deriving insights from data. With a strong proficiency in Python, R, and data visualization tools such as Power BI, Farman specializes in building scalable systems and transforming complex datasets into actionable insights that help drive business growth and enhance operational efficiency.

---

## Our Vision

**"My Work, Your Vision"**  
At Farman Karim, we believe in collaboration – turning your ideas into reality. Our commitment to excellence and continuous improvement drives us to create robust Odoo solutions tailored to meet individual client needs.

---

## Services Offered

- **Odoo Development:** Custom module creation and integration of third-party applications.
- **Data Analysis:** Leveraging various tools to unveil insights that guide business decisions.
- **Project Management:** Coordinating development teams and managing the entire project lifecycle from requirement gathering to deployment.

---

## Clients & Projects

Farman has played pivotal roles in various organizations, including:
- **Damotic Technologies:** Currently developing tailored Odoo projects like a Parcel Management System and a Ride Management Platform.
- **SyntecX Limited:** Led Odoo implementations and developed custom modules to enhance client operations.
- **Galaxy Solutions ITC:** Collaborated with cross-functional teams to ensure seamless execution of Odoo projects.

---

## Education

- **MS in Business Intelligence** (Sept 2023 – Aug 2025)  
  Université Marie-et-Louis-Pasteur, Besançon, France
- **Bachelor in Computer Science** (Sept 2017 – July 2021)  
  University of Management and Technology, Lahore, Pakistan

---

## Company Culture

Farman embraces a culture of **innovation, collaboration, and continuous growth**. We believe that a supportive workplace fueled by shared goals leads to greater achievements. Our team is encouraged to pursue professional development, balance teamwork with individual responsibilities, and foster an environment of mutual respect and creativity.

---

## Careers with Farman Karim

We are always on the lookout for passionate individuals to join our team. If you share our enthusiasm for technology and development and strive to make a difference, reach out to us! 

**Contact Information:**  
- **Call:** +33 76 96 214 73  
- **Email:** faruhaikal@gmail.com  

--- 

**Let’s connect and explore opportunities to work together!**  
Your success is our mission. 

--- 

### Follow Us  
Stay updated on our projects and services! [Link to social profiles could be added here]

--- 

**Farman Karim - Your Partner in Technology Innovation**  


In [43]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'company page', 'url': 'https://faruhaikal.wixsite.com/iamfarman'}, {'type': 'resume page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/resume'}, {'type': 'projects page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/s-projects-basic'}, {'type': 'services page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/services-7'}, {'type': 'contact page', 'url': 'https://faruhaikal.wixsite.com/iamfarman/contact'}]}


# Farman Karim Brochure

## Who We Are

**Farman Karim** is a dedicated software developer and data analyst specializing in Odoo development. With more than 3 years of experience in the tech industry, we are passionate about building efficient solutions and transforming complex datasets into actionable insights that drive business growth and operational efficiency.

### Our Mission
Our mission is to turn your vision into reality by implementing innovative Odoo solutions tailored to meet your unique needs.

## What We Do

We offer a range of **services** including:

- **Odoo Development:** Custom solutions such as Parcel Management Systems and Ride Management Platforms, expertly designed to enhance operational workflows.
- **Data Analysis:** Utilizing tools like Python, R, and Power BI to uncover valuable insights from data and drive strategic decision-making.
  
### Our Projects
Explore our **Project Portfolio** to see how we've successfully partnered with various businesses, improving their efficiency and achieving their goals through tailored software solutions.

## Our Culture

At Farman Karim, we foster a culture of collaboration and creativity. We believe in the power of teamwork, which is reflected in our approach to project execution. Our team members enjoy a friendly and supportive environment that encourages learning and personal growth, while also engaging in extracurricular activities like volleyball and table tennis. 

### Career Opportunities

As we continue to grow, we are always on the lookout for talented individuals to join our team. If you are passionate about technology and innovation, consider exploring the career paths available at Farman Karim. We welcome applicants who are eager to contribute to exciting projects and be part of a dynamic team.

## Connect with Us!

Ready to transform your ideas into reality or learn more about our work? 

- **Email:** faruhaikal@gmail.com  
- **Phone:** +33 76 96 214 73  

Let’s collaborate and create something amazing together! 

---
*Farman Karim | Odoo Developer & Data Analyst*  
*Follow us for updates and more information!*


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>